## Include common.ipynb

In [1]:
from collections import OrderedDict
from typing import List, Tuple

import numpy as np
import torch

from flwr.common import Context, Metrics
from flwr.server import start_server, ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg

## Flower Server

In [2]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    config = ServerConfig(num_rounds=1)

    return strategy, config


strategy, config = server_fn(context=None)

start_server(
    server_address="127.0.0.1:8081",
    config=config,
    strategy=strategy
)